In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "unsloth/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", load_in_8bit=True, device_map="cpu")

In [ ]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 32.1 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

from peft import LoraConfig, get_peft_model, TaskType


In [ ]:
lora_config = LoraConfig (
    r = 8,
    lora_alpha = 32,
    target_modules = ["q_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/drive/MyDrive/bank%20query.csv")

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)


In [ ]:
dataset

In [ ]:
def tokenize(example):
  full_text = example["input"] + "\n" + example["output"]
  return tokenizer(full_text, max_length=512, truncation=True, padding_side="right")

In [ ]:
tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/1423 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
word = tokenizer("hi how are you?", return_tensors="pt").to("cuda")

In [ ]:
result

In [ ]:
trainings_args = TrainingArguments(
    per_device_train_batch_size=4,
    output_dir="./llama",
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    warmup_ratio=0.03,
    logging_steps=10,
    save_strategy="epoch"
)

In [ ]:
from transformers import DataCollatorForLanguageModeling # Import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    args = trainings_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("retail_banking_adapter")